# `TabuLLM`: Feature Extraction from Tabular Data Text using Large Language Models (LLMs) 

## Introduction

Text embedding is the process of converting natural-language text (a single word or an entire document) to a numeric vector, such that semantically similar text are mapped to nearby points in the numeric vector space. Embeddings are often used in applications such as information retrieval and search engines, text classification (including sentiment analysis), Q&A chatbots and recommender systems.

Embeddings can also be used as features in predictive models, an application that hasn't received as much attention. Recently, it has been shown that modern LLMs can not only extract useful predictive information from domain-specific text, but they may now exceed domain experts in predictive accuracy (Sharabiani et al, 2024). The `TabuLLM` Python package provides various functionalities to support the use of LLMs for extracting numeric features from text columns in tabular data, and for incoporating these embeddings in predictive models alongside other features.

In particular, `TabuLLM` consists of four modules:
1. `embed` - Unified interface for converting one or more text column(s) in the data to a numeric matrix, using commercial LLMs (OpenAI, Google Vertex AI), open-source LLMs (available on the Hugging Face model respository, and accessed via the [sentence transformers](https://sbert.net/) package), as well as earlier-generation embedding methods such as [Doc2Vec](https://radimrehurek.com/gensim/models/doc2vec.html).
1. `cluster` - Python implementation of [spherical k-means](https://www.jstatsoft.org/article/view/v050i10), which can be used for grouping data points according to the embedding vectors produced by LLMs. Since embeddings only contain directional information and their magnitude is not meaningful, it is more appropriate to use spherical k-means, which replaces the Euclidean distance - used in standard k-means - with [cosine distance](https://en.wikipedia.org/wiki/Cosine_similarity). (Note: While the cluster-assignment step in the Lloyd's algorithm for esitmating k-means would be identical using Euclidean vs. cosine distance metrics, the centroid-update step would be different since taking a simple average of L2-normalized vectors does not produce another L2-normalized vector.)
1. `explain` - 1) Prompt generation for soliciting descriptive labels for data clusters (such as those generating from the embedding vectors, as discussed above), 2) Wrapper for interacting with text-completion LLMs (currently: OpenAI and Google).
1. `compress` - Applying k-nearest-neighbor - in supervised mode and wrapped in cross-fit - to compress the high-dimensional embedding matrix into a single feature column. Cross-fit helps minimize overfitting, by making the in-sample predictions of KNN resemble out-of-sample data. The compressed feature can subsequently be used alongside other features in the ultimate predictive model.

The rest of this tutorial will describe each module in more detail, and illustrate their use via a running example. It is, therefore, useful to start a brief overview of the dataset that will be used throughout the tutorial.

## Dataset: Acute Kidney Injury following Pediatric Cardiopulmonary Bypass

### Overview of Data Columns

The data used in the `TabuLLM` tutorials is an excerpt from the dataset introduced in Sharabiani et al (2024), collected during 2019-2021 from pediatric CPB surgeries in the UK. Columns included:
- `is_female`: Patient gender (`0`: male, `1`: female)
- `age`: Patient age at the time of operation (years)
- `height`: Patient height at the time of operation (cm)
- `weight`: Patient weight at the time of operation (kg)
- `optime`: Length of operation (minutes)
- `diagnoses`: An ordered collection of medical diagnosis codes for the patient (starting with the primary diagnosis), each including a numeric value followed by a text description of the code. Multiple codes are separated by semicolons.
- `operations`: Similar to `diagnoses`, but representing the procedures performed on the patient during CPB.
- `cr_ratio_log`: Logarithm of postoperative to preoperative creatinine ratio in the patient. This is one of the two metrics that is used in defining postoperative acute kidney injury (AKI) by [KDIGO](https://kdigo.org/), a 'global organization developing and implementing evidence-based clinical practice guidelines in kidney disease'. This will be used as a continuous outcome variable in the tutorials (regression).
- `aki_severity`: Severity of postoperative acute kidney injury (AKI). This is a binarized version of the KDIGO score that can take the values 0, 1, 2, 3. The severity score is 0 if the original score is 0 or 1, and 1 if the original score is 2 or 3.

Let's load and explore the data:

In [4]:
import pandas as pd
df = pd.read_csv('../data/raw.csv')
df.head()

,is_female,age,height,weight,optime,diagnoses,operations,cr_ratio_log,aki_severity
0,1,18.11,148,80.9,112,155500. Cardiac conduit complication;010125. P...,123610. Replacement of cardiac conduit,-0.014599,0
1,1,18.23,169,56.1,144,091591. Aortic regurgitation;091519. Congenita...,121791. Aortic root replacement: valve sparing...,0.366463,1
2,1,16.86,166,61.6,114,155516. Cardiac conduit failure;090101. Common...,123610. Replacement of cardiac conduit;123452....,0.084899,0
3,1,16.88,162,44.3,109,010116. Partial anomalous pulmonary venous con...,120017. Scimitar syndrome (partially anomalous...,0.168335,0
4,0,18.12,175,70.5,119,155516. Cardiac conduit failure;010133. Left h...,123610. Replacement of cardiac conduit,0.308301,0


You can see that each diagnosis code is a number followed by its description. The order of entry is medically informative, reflecting the primary vs. secondary diagnoses. Note that, given the standardized nature of the medical codes, we could alternatively use a Bag-of-Codes (BoC) approach for encoding this column, where we would assign a dummy binary to each medical code. (Of course, this would lose the information contained in the order of codes.) See Sharabiani et al (2024) for a comparison of this BoC encoding method against alternatives, including LLM-based embedding algorithms which are the focus of the `TabuLLM` package.

## The `Embed` Module

The workhorse of this module is the `TextColumnTransformer` class, which implements `scikit-learn`'s transformer interface, namely the `fit`, `transform` and `fit_transform` methods. The most important argument to the class constructor is `type`, which specifies the type of embedding algorithm to be used. As of this writing (August 2024), the available options are `openai`, `google`, `st` (open-source Hugging Face models accessed via the [`sentence-transformers`](https://sbert.net/) package), and `doc2vec` (via [`gensim`](https://radimrehurek.com/gensim/models/doc2vec.html) package). With the exception of `doc2vec`, the remaining models do not train on the data, which means the `fit` function is simply a pass-through for them.

Let's briefly look at how each type of model can be used.

### OpenAI

The `TextColumnTransformer` class constructor expects two arguments specific to OpenAI: `openai_client` and `embedding_model_openai`. The following code shows how to instantiate an OpenAI client:

In [5]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=openai_api_key)

It is assumed that the user has a valid account with OpenAI and obtained an API key, which is saved as the `OPENAI_API_KEY` environment variable in a `.env` file, which is read by `load_dotenv()`. `embedding_model_openai` must be one of the strings listed on [OpenAI's website](https://platform.openai.com/docs/guides/embeddings/embedding-models). The default is the largest and most accurate model, `text-embedding-3-large`. We can now instantiate and use `TextColumnTransformer`. (NOTE: Running the following code would call OpenAI's embedding model and thus incur a small cost.)

In [13]:
from TabuLLM.embed import TextColumnTransformer
obj = TextColumnTransformer(type = 'openai', openai_client = client, embedding_model_openai = 'text-embedding-3-small')
X = obj.fit_transform(df.loc[:5, ['diagnoses']])
print(X.shape)

(6, 1536)


Note that we must pass only the text column(s) of interest - as a `pandas` dataframe - to the `fit_transform` function. If multiple text columns are provided, they will be concatenated into a single text column, using the value of the argument `colsep` (which defaults to ` || `).

`fit_transform` returns a `pandas` dataframe. While the number of rows is the same as the input dataframe, the number of columns is determined by the embedding model being called. For instance, the embedding vector returned by `text-embedding-3-small` has a length of 1536, as seen above. On the other hand, embeddings returned by OpenAI's latest model, `text-embedding-3-large` have twice the length, i.e., 3072.

Let's examine column names of the returned dataframe:

In [16]:
print(X.columns[:5])

Index(['X_0', 'X_1', 'X_2', 'X_3', 'X_4'], dtype='object')


The prefix is controlled by the argument `return_cols_prefix` to the class constructor (default is `X`, as seen above).

### Google (Vertex AI)

To use Google's embedding models, we must have a valid Google Cloud Platform account, including a Vertex AI project id and location. As with OpenAI, these can be loaded as environment variables:

In [17]:
google_project_id = os.getenv('VERTEXAI_PROJECT')
google_location = os.getenv('VERTEXAI_LOCATION')

X = TextColumnTransformer(
    type = 'google'
    , google_project_id = google_project_id
    , google_location = google_location
).fit_transform(df.loc[:5, ['diagnoses']])
print(X.shape)

(6, 768)


We can see that here the Google embeddings have a length of 768, i.e., half of the output from the OpenAI model tested earlier. As with OpenAI, Google also accepts a model type, which we can specify via `embedding_model_google`. Valid options can be found [here](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/text-embeddings-api). In addition, users can also provide `google_task`, which specifies what downstream application the embeddings will be used for. Options and their descriptions can be found [here](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/text-embeddings-api). Default in `TextColumnTransformer` is `SEMANTIC_SIMILARITY`.

### Sentence Transformers

Besides the commercial embedding models provided by OpenAI and Google, we can also use the open-source LLMs that are checked into the Hugging Face platform. The `sentence-transformers` Python package provides a convenient wrapper to download and use these models locally. In addition to using `type = 'st'`, we must also specify `embedding_model_st`. This can be found by browsing to the model's homepage on Hugging Face. Below is an example call:

In [19]:
X = TextColumnTransformer(
    type = 'st'
    , embedding_model_st = 'sentence-transformers/all-MiniLM-L6-v2'
).fit_transform(df.loc[:5, ['diagnoses']])
print(X.shape)

c:\Users\alire\anaconda3\envs\devTEFE\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


(6, 384)


It is important to note that, unlike the case with OpenAI and Google where inference (text embedding, to be more precise) occurs in the cloud, sentence transformer models are downloaded to your machine and run locally. This is especially important when using large models with many parameters, since they require a nontrivial amount of space to store all the weights of their deep neural network, and also performing inference on these networks can be time consuming, unless using accelerators such as GPUs. (The above model, `sentence-transformers/all-MiniLM-L6-v2` was chosen specifically for this example since it is a distilled - very small - model.)

An advantage of using sentence transformer models, in addition to being free to use, is that they can be customized for the specific needs of our problem. For instance, we can fine-tune these embedding models on data specific to our application domain. A thorough discussion of fine-tuning is beyond the scope of this tutorial.

### Doc2Vec

The last type of embedding model included in `TabuLLM` is Doc2Vec (Le and Mikolov, 2014), which is an extension of word2vec. `TextColumnTransformer` uses the implementation of Doc2Vec provided in `gensim`. This algorithm has an important distinction from the the others mentioned above: Doc2Vec requires training before it can be used in inference, i.e., it is not a pretrained model. As such, there are a two important hyperparameters for Doc2Vec that users must select in `TabuLLM`:
1. `doc2vec_vector_size`: Length of output embedding vectors (default: 10),
1. `doc2vec_epochs`: Number of training epochs on the data (default: 40).

An example call would be:

In [21]:
X = TextColumnTransformer(
    type = 'doc2vec'
    , doc2vec_epochs = 20
    , doc2vec_vector_size = 50
).fit_transform(df.loc[:5, ['diagnoses']])
assert X.shape == (6, 50)

## Handling of Multiple Text Columns

In our examples so far, we focused on one of the two text fields, `diagnoses`, provided in the data. However, we have a second one as well, `operations`. There are several options for dealing with multiple text columns. For example:
1. Perform embedding and all downstream feature-extraction (e.g., `cluster` or `compress`) steps on each text column separately. Form a union of all the features in the final step, i.e., the predictive model.
1. Concatenate the text columns into a single one, and perform all downstream transformations, starting with `embed`, on this concatenated column.

There are also options in between the above two. For instance, we can `embed` each text column independently, form a union of embedding vectors, and `compress` the combined embedding vectors.

To provide user convenience, `TextColumnTransformer` contains option to consolidate and embed multiple text columns. Concatenated columns will be separated by the string `colsep`, which defaults to `' || '`. All the user has to do is include all text columns in the input dataframe:

In [5]:
X = TextColumnTransformer(
    model_type = 'st'
).fit_transform(df.loc[:5, ['diagnoses', 'operations']])
print(X.shape)

c:\Users\alire\anaconda3\envs\devTEFE\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\alire\anaconda3\envs\devTEFE\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


(6, 384)


We see that, since we combined the two text columns prior to embedding, the number of columns for resulting feature matrix is unchanged.

It is also possible, while combining multiple text columns, to prefix each column with a descriptive text to provide more context to the LLM for embedding. For instance, we can prepend the words 'patient diagnoses:' and 'CPB procedures:' to each column before embedding:

In [13]:
df['diagnoses_and_operations'] = 'Patient diagnoses: ' + df['diagnoses'] + ' || CPB procedures: ' + df['operations']
list(df.loc[:5, 'diagnoses_and_operations'])

["Patient diagnoses: 155500. Cardiac conduit complication;010125. Pulmonary atresia + VSD + 'MAPCA'(s) || CPB procedures: 123610. Replacement of cardiac conduit",
 'Patient diagnoses: 091591. Aortic regurgitation;091519. Congenital anomaly of aortic valve;071205. Doubly committed juxta-arterial ventricular septal defect (VSD) with anteriorly malaligned fibrous outlet septum and perimembranous extension || CPB procedures: 121791. Aortic root replacement: valve sparing technique',
 'Patient diagnoses: 155516. Cardiac conduit failure;090101. Common arterial trunk;110021. Cardiac arrest || CPB procedures: 123610. Replacement of cardiac conduit;123452. Pacemaker system placement: biventricular',
 'Patient diagnoses: 010116. Partial anomalous pulmonary venous connection of Scimitar type || CPB procedures: 120017. Scimitar syndrome (partially anomalous pulmonary venous connection) repair',
 'Patient diagnoses: 155516. Cardiac conduit failure;010133. Left heart obstruction at multiple sites (i

In [10]:
X = TextColumnTransformer(
    model_type = 'st'
).fit_transform(df.loc[:5, ['diagnoses_and_operations']])
print(X.shape)

c:\Users\alire\anaconda3\envs\devTEFE\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


(6, 384)


## Recap

To recap, the `TextColumnTransformer` class in the `Embed` module of `TabuLLM` provies a unified interface for applying various text embedding algorithms to text columns in tabular data. As it implements the `scikit-learn` transformer interface, it can be used as a component in composite estimators and complex predictive pipelines. Other tutorials will provide examples of doing so.

## References

1. Sharabiani, Mansour TA, et al. "Predictive and Explainable Analysis of Post-operative Acute Kidney Injury in Children undergoing Cardiopulmonary Bypass: An Application of Large Language Models." medRxiv (2024): 2024-05.
1. Le, Quoc, and Tomas Mikolov. "Distributed representations of sentences and documents." International conference on machine learning. PMLR, 2014.